In [58]:

from sympy import symbols, Eq, And, Xor
from sympy.logic.inference import satisfiable
import re

# Define symbols for houses
houses = 3
house_colours = symbols(f'house_colour:{houses}')
house_owners = symbols(f'house_owner:{houses}')
house_drinks = symbols(f'house_drink:{houses}')

# Specific entities
colours = (blue, green, red, yellow, purple) = symbols('blue green red yellow purple')
nationalities = (canadian, american, japanese, british, german) = symbols('canadian american japanese british german')
drinks = (coffee, milk, tea, juice, water) = symbols('coffee milk tea juice water')

# now for constraints
constraints = []

# 1:1 house:nationality
# each house owned by one nationality
for h_owner in house_owners[:houses]:
    constraints.append(Xor(*[Eq(h_owner, nationality) for nationality in nationalities[:houses]]))
# each nationality  is assigned to exactly one house
for nationality in nationalities[:houses]:
    constraints.append(Xor(*[Eq(h_owner, nationality) for h_owner in house_owners[:houses]]))

# 1:1 house:colour
# each house is assigned one colour
for h_colour in house_colours[:houses]:
    constraints.append(Xor(*[Eq(h_colour, colour) for colour in colours[:houses]]))
# each colour is assigned to one house
for colour in colours[:houses]:
    constraints.append(Xor(*[Eq(h_colour, colour) for h_colour in house_colours[:houses]]))

# 1:1 house:drink
# each house is assigned one drink
for h_drink in house_drinks[:houses]:
    constraints.append(Xor(*[Eq(h_drink, drink) for drink in drinks[:houses]]))
# each drink is assigned to one house
for drink in drinks[:houses]:
    constraints.append(Xor(*[Eq(h_drink, drink) for h_drink in house_drinks[:houses]]))
    
# the blue house has milk
constraints.append(Xor(*[(Eq(house_colours[i], blue) & Eq(house_drinks[i], milk)) for i in range(houses)]))

# only the canadian owns the house which has coffee
constraints.append(Xor(*[(Eq(house_owners[i], canadian) & Eq(house_drinks[i], coffee)) for i in range(houses)]))

# Combine constraints into one logical expression using And
combined_constraints = And(*constraints)

# Check satisfiability
result = satisfiable(combined_constraints)
houses = {}

def print_result(result):
    # Process each entry in the SymPy result
    for attr, value in [(k.lhs, k.rhs) for k,v in result.items() if v]:
        # Extract the attribute type and house number from the symbol name
        attr_name = str(attr)
        if 'owner' in attr_name:
            attr_type = 'owner'
        elif 'drink' in attr_name:
            attr_type = 'drink'
        elif 'colour' in attr_name:
            attr_type = 'colour'
        house_num = int(re.search(r'(\d+)$', attr_name).group(1))
        
        # Initialize the house entry if not already present
        if house_num not in houses:
            houses[house_num] = {"owner": None, "drink": None, "colour": None}
        
        # Assign the value to the corresponding attribute
        houses[house_num][attr_type] = value

    # Format and print the results
    for house_num in sorted(houses):
        owner = houses[house_num]["owner"]
        colour = houses[house_num]["colour"]
        drink = houses[house_num]["drink"]
        print(f"The {owner} owns the {colour} house and drinks {drink}")

if result: 
    print_result(result)
else:
    print("No solution found")

The american owns the blue house and drinks milk
The canadian owns the green house and drinks coffee
The japanese owns the red house and drinks tea
